In [ ]:
#Import Required libraries
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import sys
from textblob import TextBlob
import pandas as pd
from ibmdbpy import IdaDataBase, IdaDataFrame

In [ ]:
# Function to estimate tweet sentiment.
# Poor man's version using TextBlob library

def tweetSentiment(text) :
    tweetSentiment = list(TextBlob(text).sentiment)
    polarity = tweetSentiment[0]
    if polarity <= 0.10 and polarity >= -0.10 :
        return "neutral"
    elif polarity > 0 :
        return "positive"
    else :
        return "negative"

In [ ]:
# Seperate out sentiment polarity
def tweetPolarity(text) :
    tweetPolarity = list(TextBlob(text).sentiment)
    tweetPolarity = tweetPolarity[0]
    return tweetPolarity

In [ ]:
# Seperate out sentiment subjectivity
def tweetSubjectivity(text) :
    tweetSubjectivity = list(TextBlob(text).sentiment)
    tweetSubjectivity = tweetSubjectivity[1]
    return tweetSubjectivity

In [ ]:
# Create a streamer object
class StdOutListener(StreamListener):
    
    def __init__(self, api = None):
        
        super(StdOutListener, self).__init__()
        self.idadb_46fb2f94269f47ae943466453b10e5e5 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=YourDashDBHostURL;Port=50000;PROTOCOL=TCPIP;UID=dash5040;PWD=YourDashDBPassword')
        self.data_df_1 = IdaDataFrame(self.idadb_46fb2f94269f47ae943466453b10e5e5, 'DASH5040.TWEETS')
        
        # Define a function that is initialized when the miner is called
        # That sets the api
        self.api = api


    # When a tweet appears, append it to a dataframe.
    def on_status(self, status):
        # If the tweet is not a retweet
        if True:
            # Try to
            try:
                # Creates Panda
                df = pd.DataFrame([[status.id_str,
                                    status.created_at,
                                    status.coordinates,
                                    status.coordinates,
                                    status.source,
                                    status.text,
                                    tweetSentiment(status.text),
                                    status.lang,
                                    status.favorite_count,
                                    status.user.location,
                                    status.user.time_zone,
                                    status.retweet_count,
                                    status.user.created_at,
                                    status.user.id,
                                    status.lang,
                                    status.user.favourites_count,
                                    status.user.followers_count,
                                    status.user.following,
                                    status.user.name,
                                    status.user.profile_background_color,
                                    status.user.profile_image_url,
                                    status.user.screen_name,
                                    status.user.protected,
                                    status.user.statuses_count]], columns = ['id_str',
                            'created_at',
                            'coordinates-x',
                            'coordinates-y',
                            'source',
                            'text',
                            'sentiment',
                            'lang',
                            'favorite_count',
                            'user_location',
                            'user_timezone',
                            'retweet_count',
                            'user_created_at',
                            'user_id_str',
                            'user_lang',
                            'user_favourites_count',
                            'user_followers_count',
                            'user_following_count',
                            'user_name',
                            'user_background_color',
                            'user_profile_image_url',
                            'user_profile_screen_name',
                            'user_tweets_protected',
                            'user_statuses_count'])
                self.idadb_46fb2f94269f47ae943466453b10e5e5.append(self.data_df_1, df)
                self.idadb_46fb2f94269f47ae943466453b10e5e5.commit()

                ### so data /received/ here
                ### if print is used here will output everything. Consider running an append here?
            
                
            # If some error occurs
            except Exception as e:
                # Print the error
                print(e)
                # and continue
                pass
        
        # Return nothing
        return

    # When an error occurs
    def on_error(self, status_code):
        # Print the error code
        print('Encountered error with status code:', status_code)

        # If the error code is 401, which is the error for bad credentials
        if status_code == 401:
            # End the stream
            return False

    # When a deleted tweet appears
    def on_delete(self, status_id, user_id):

        # Print message
        print("Delete notice")

        # Return nothing
        return

    # When reach the rate limit
    def on_limit(self, track):

        # Print rate limiting error
        # This is the error message you will see if you search for any hot trend in the Twitter universe
        print("Rate limited, continuing")

        # Continue mining tweets
        return True

    # When timed out
    def on_timeout(self):

        # Print timeout message
        print(sys.stderr, 'Timeout...')

        # Wait 10 seconds
        time.sleep(10)

        # Return nothing
        return

In [ ]:
# Create a mining function
def start_mining(queries):

    #Variables that contains the user credentials to access Twitter API
    # These keys/tokens are for my personal Twitter account.  Please use them only
    # for tweet retrieval and not posting of tweets.
    consumer_key = "Your consumer_key"
    consumer_secret = "Your consumer_secret"
    access_token = "Your access_token"
    access_token_secret = "Your access_token_secret"

    # Create a listener
    l = StdOutListener()

    # Create authorization info
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Create a stream object with listener and authorization
    stream = Stream(auth, l, encoding='en')
    # stream = Stream(auth, l)

    # Run the stream object using the user defined queries
    # "track" is specifying how I want to listen on Twitter, here I will be listening for whatever is assigned to "queries"
    # Languages is set to "en" because I don't want to deal with non english "as indicated by the author of the tweets" tweets.
    stream.filter(track=queries, languages=['en'])

In [ ]:
# Identify string for Twitter search.  "Summer" works well, it provides a steady stream of tweets but it doesn't exceed Twitter
# limits on consuption.  We are not paying to mine Twitter so they restrict how many tweets/unit time you can pull down.
# If you want to see how the limitations work then search for "trump", there is always a lot of activity around this string.
# Start the miner
# start_mining(['trump']) # Will produce rate limit error
start_mining(['Summer']) # Will produce a nice steady stream without rate limit error
# start_mining(['grapes','apples', 'cake','carlyiscoolasice']) # Search for "grapes" and "apples"
# start_mining(['tacos']) # Will produce rate limit error

